In [1]:
conda install selenium

Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [2]:
conda install pymongo

Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [123]:
from wine_setup import WineSetup

dessert_wine_url = "https://www.vivino.com/explore?e=eJzLLbI1VMvNzLM1UMtNrLA1NTBQS660DQ1WSwYSLmoFQNn0NNuyxKLM1JLEHLX8JNuixJLMvPTi-OT80rwStXzblNTiZLXykuhYW3Og1mIgXVqsVmybnAgAVJoe1g=="
dessert_wine = WineSetup(dessert_wine_url)
dessert_wine.setup_driver()
dessert_wine.run_timestamp()
dessert_wine.run_sel_lazyloading('Dessert')
dessert_wine.run_timestamp()

Current Timestamp:  05-Feb-2020 (17:25:26.762094)
Total Number:  797
Curr Row Number:  200
Curr Row Number:  400
Curr Row Number:  600
Current Timestamp:  05-Feb-2020 (17:30:44.285610)


In [124]:
dessert_rows = dessert_wine.create_rows()

In [125]:
len(dessert_rows)

797

In [77]:
%reload_ext autoreload
%autoreload 2

In [128]:
import copy

# austria_wine_df = pd.DataFrame()
all_rows = []

# Let's store each row as a dictionary 
empty_row = {
    "title": None, "location": None, "country": None, "price": None, "type": None, "ratings": None, "num_ratings": None, "reviews": None, "url": None
}

for row in dessert_rows:
    new_row = copy.copy(empty_row)
    # A list of all the entries in the row.
    new_row['title'] = row.find("span", {"class": "vintageTitle__wine--U7t9G"}).text
    location = row.find("div", {"class": "vintageLocation__vintageLocation--1DF0p"})
    new_row['location'] = location.findChildren()[-1].text
    country = row.find("i", {"class": "vintageLocation__countryFlag--1HbXr"})['title']
    new_row['country'] = country
    price_button = row.find("button", {"class": "addToCartButton__addToCartButton--qZv9F"})
    if price_button:
        new_row['price'] = (float(price_button.find("span").text.replace("$", "")))
    new_row['type'] = 'Dessert'
    new_row['ratings'] = row.find("div", {"class": "vivinoRatingWide__averageValue--1zL_5"}).text
    new_row['num_ratings'] = int(row.find("div", {"class": "vivinoRatingWide__basedOn--s6y0t"}).text.split()[0])
    review_div = row.find("div", {"class": "review__note--2b2DB"})
    if new_row['title'] == "Paladini Vineyard Royal Escort Vintage Port N.V.":
        print(row)
    if review_div:
        new_row['reviews'] = review_div.text
    clean_div = row.find("div", {"class": "cleanWineCard__cleanWineCard--tzKxV cleanWineCard__row--CBPRR"})
    if clean_div:
        new_row['url'] = 'https://www.vivino.com' + clean_div.find("a")['href']
    all_rows.append(new_row)

In [129]:
all_rows

[{'title': 'Dolce (Late Harvest) N.V.',
  'location': 'Napa Valley',
  'country': 'United States',
  'price': None,
  'type': 'Dessert',
  'ratings': '4.5',
  'num_ratings': 1437,
  'reviews': None,
  'url': 'https://www.vivino.com/dolce-winery-dolce-late-harvest/w/1316060?year=N.V.'},
 {'title': 'Angelica 2010',
  'location': 'California',
  'country': 'United States',
  'price': None,
  'type': 'Dessert',
  'ratings': '4.5',
  'num_ratings': 277,
  'reviews': None,
  'url': 'https://www.vivino.com/v-sattui-winery-angelica/w/1782940?year=2010'},
 {'title': 'Dolce (Late Harvest) 2012',
  'location': 'Napa Valley',
  'country': 'United States',
  'price': None,
  'type': 'Dessert',
  'ratings': '4.5',
  'num_ratings': 207,
  'reviews': None,
  'url': 'https://www.vivino.com/dolce-winery-dolce-late-harvest/w/1316060?year=2012'},
 {'title': 'Dolce (Late Harvest) 2008',
  'location': 'Napa Valley',
  'country': 'United States',
  'price': None,
  'type': 'Dessert',
  'ratings': '4.5',
  'n

In [130]:
import pymongo
from pymongo import MongoClient

client = MongoClient('localhost', 27017)
wine_db = client['wine_db']
wines = wine_db['wines']

In [131]:
for row in all_rows:
    wines.insert_one(row)